##### Copyright 2024 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemma - Game Design Brainstorming

This cookbook illustrates how you can employ the instruction-tuned model version of Gemma as a chatbot to aid in your game design process by fostering brainstorming sessions effectively.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/[Gemma_2]Game_Design_Brainstorming.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Setup

### Select the Colab runtime
To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model. In this case, you can use a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.


### Gemma setup on Kaggle
To complete this tutorial, you'll first need to complete the setup instructions at [Gemma setup](https://ai.google.dev/gemma/docs/setup). The Gemma setup instructions show you how to do the following:

* Get access to Gemma on kaggle.com.
* Select a Colab runtime with sufficient resources to run the Gemma 2B model.
* Generate and configure a Kaggle username and API key.

After you've completed the Gemma setup, move on to the next section, where you'll set environment variables for your Colab environment.

### Set environment variables

Set environment variables for `KAGGLE_USERNAME` and `KAGGLE_KEY`.

In [2]:
import os
os.environ["kaggle_username"] = "pavit0040604nis"


In [3]:
import os
from google.colab import userdata

os.environ["KERAS_BACKEND"] = "jax"  # Or "tensorflow" or "torch".

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["kaggle_username"] = "pavit0040604nis"
os.environ["kaggle_key"] = "fd2b0f46519b653d6ffe018d1dc1666d"

### Install dependencies

Install Keras and KerasNLP.

In [4]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.3/731.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.4 MB/s eta 0:00:00


## Core Concept

In [5]:
# A super concise description of your game
pitch = "In the heart of a reimagined Joseon Dynasty, where steam-powered machines and clockwork contraptions reign supreme, embark on an epic adventure to unravel the mysteries of a forgotten past and shape the destiny of a nation on the brink of revolution."

In [6]:
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [7]:
import os
import shutil

# Define the target directory
kaggle_dir = "/root/.kaggle/"

# Create the directory if it doesn't exist
os.makedirs(kaggle_dir, exist_ok=True)

# Move kaggle.json to the correct location
shutil.move("kaggle.json", os.path.join(kaggle_dir, "kaggle.json"))

# Set the correct permissions
os.chmod(os.path.join(kaggle_dir, "kaggle.json"), 600)

print("✅ Kaggle API key set up successfully!")


✅ Kaggle API key set up successfully!


## Load Model

In [8]:
import keras
import keras_nlp

# Run at half precision to fit in memory
keras.config.set_floatx("bfloat16")

import textwrap
from IPython.display import Markdown


def to_markdown(text):
    text = text.replace("•", "  *")
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))

gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_2b_en")
gemma_lm.compile(sampler="top_k")


class ChatState():
  """
  Manages the conversation history for a turn-based chatbot
  Follows the turn-based conversation guidelines for the Gemma family of models
  documented at https://ai.google.dev/gemma/docs/formatting
  """

  __START_TURN_USER__ = "<start_of_turn>user\n"
  __START_TURN_MODEL__ = "<start_of_turn>model\n"
  __END_TURN__ = "<end_of_turn>\n"

  def __init__(self, model, system=""):
    """
    Initializes the chat state.

    Args:
        model: The language model to use for generating responses.
        system: (Optional) System instructions or bot description.
    """
    self.model = model
    self.system = system
    self.history = []

  def add_to_history_as_user(self, message):
      """
      Adds a user message to the history with start/end turn markers.
      """
      self.history.append(self.__START_TURN_USER__ + message + self.__END_TURN__)

  def add_to_history_as_model(self, message):
      """
      Adds a model response to the history with start/end turn markers.
      """
      self.history.append(self.__START_TURN_MODEL__ + message)

  def get_history(self):
      """
      Returns the entire chat history as a single string.
      """
      return "".join([*self.history])

  def get_full_prompt(self):
    """
    Builds the prompt for the language model, including history and system description.
    """
    prompt = self.get_history() + self.__START_TURN_MODEL__
    if len(self.system)>0:
      prompt = self.system + "\n" + prompt
    return prompt

  def send_message(self, message):
    """
    Handles sending a user message and getting a model response.

    Args:
        message: The user's message.

    Returns:
        The model's response.
    """
    self.add_to_history_as_user(message)
    prompt = self.get_full_prompt()
    response = self.model.generate(prompt, max_length=4096)
    result = response.replace(prompt, "")  # Extract only the new response
    self.add_to_history_as_model(result)
    return result

  def show_history(self):
    for h in self.history:
      print(h)


chat = ChatState(gemma_lm)

100%|██████████| 782/782 [00:00<00:00, 429kB/s]


100%|██████████| 4.87G/4.87G [00:55<00:00, 94.8MB/s]


100%|██████████| 591/591 [00:00<00:00, 1.15MB/s]


100%|██████████| 4.04M/4.04M [00:00<00:00, 104MB/s]


## Elaborate with the given core game concept

In [9]:
result = chat.send_message(
    f"Elaborate about this game with the given core concept below.\n{pitch}"
)
to_markdown(result)

> ## The Clockwork Shogun: A Game of Innovation & Rebellion
> 
> **Concept:** A blend of action RPG, strategy, and historical fiction set in a Joseon dynasty where steam and clockwork have become commonplace. The story centers around a young apprentice inventor tasked with uncovering the secrets of a mysterious past, all while navigating political intrigue and a looming revolution.
> 
> 
> **Gameplay:** 
> 
> * **Action RPG:**  Players control characters with unique abilities and skills, engaging in fast-paced combat using steam-powered gadgets (think steampunk-era melee weapons and firearms).  Combat will be a blend of traditional melee and ranged combat with an emphasis on timing and strategic positioning.
> * **Resource Management & Crafting:**  Players will manage their own workshops and factories, crafting new inventions and weapons, upgrading existing ones, and utilizing them in battles or exploration.  Resource scarcity and strategic production will be key to success.
> * **Social Exploration & Diplomacy:**  The game will feature a robust social simulation system, with the ability to interact with various factions and characters. Players will need to navigate a complex web of political alliances, build relationships, and even manipulate events for their gain.
> * **Puzzle Solving & Exploration:**  Beyond combat, players must solve intricate puzzles using the unique mechanics and technologies of their world. Exploration will lead them to ancient secrets, lost technologies, and hidden clues.
> * **Choices & Consequences:**  The player’s decisions will have a significant impact on the story, shaping the fate of the Joseon Dynasty and the revolution. 
> 
> **Story:**
> 
> The game will follow the protagonist:
> 
> * **A young, gifted apprentice inventor:**  Their journey will be a quest to understand their family's legacy and the mysterious technology that has been passed down for generations. 
> * **The "Shogun" (a title with a double meaning):** The Shogun's legacy and the secrets of the past are intertwined with the protagonist's destiny, creating a compelling narrative arc where the player must choose between the old ways and the future. 
> 
> **Key Features:**
> 
> * **Dynamic and Engaging World:** A richly detailed world that combines the aesthetics of Joseon with the technological innovations of the future.  Imagine bustling city streets powered by steam, ornate clockwork automatons, and hidden workshops brimming with innovation.
> * **Intriguing Political System:** Players will need to navigate a complex system of courtly intrigue, political factions, and social hierarchy, where alliances can shift quickly.
> * **Moral Dilemmas:**  The player will face tough choices with far-reaching consequences, forcing them to grapple with the complexities of power, revolution, and technological advancements.
> * **Immersive Storytelling:**   A captivating narrative driven by rich lore, memorable characters, and engaging dialogues, all set against the backdrop of a turbulent historical landscape.
> * **Multiplayer Options:**  Players can collaborate in crafting new inventions, trading resources, or even engaging in collaborative missions.  
> 
> **Overall:** 
> 
> "The Clockwork Shogun" aims to offer a unique and engaging experience that blends history, action, strategy, and a touch of fantasy.  This world offers an opportunity to explore what could have happened if technology took a different turn, while also delving into the complexities of social change and the power of individual choice during a pivotal time in history. 
> 
> 
> **Visuals:** 
> 
> * **A blend of realistic and fantastical:**  A visually stunning world with detailed character portraits, intricate cityscapes powered by steam, and awe-inspiring clockwork contraptions.  
> * **Distinct Art Style:**  A unique and evocative aesthetic that captures the essence of the Joseon Dynasty and the futuristic steampunk elements. 
> * **Atmospheric Soundtrack:**  A score that complements the game's narrative and atmosphere, blending traditional Korean music with modern electronic and orchestral elements. 
> <end_of_turn>

## Main Characters

In [10]:
result = chat.send_message("Design main characters.")
to_markdown(result)

> Here are some character concepts for "The Clockwork Shogun," designed with a blend of historical context and steampunk flair: 
> 
> **Protagonist:** **Jun-seo**
> 
> * **Age:** 18-20 years old
> * **Background:**  A young, bright-eyed apprentice inventor from a family of respected artisans. They've been taught traditional craft but have always felt drawn to the possibilities of steam and clockwork. 
> * **Personality:**  Curiosity-driven, idealistic, a natural tinkerer with a touch of recklessness. They are passionate about their work and driven to uncover their family's lost secrets, believing they hold the key to a better future. 
> * **Skills:**  Excellent with steam-powered gadgets, quick learner, resourceful inventor. They are adept at improvising and creating new inventions. 
> * **Visual:**  Lean and agile, with dark, expressive eyes and tousled dark brown hair. They favor practical clothing with pockets for tools and gadgets.  
> 
> **Mentor:** **Han-seok**
> 
> * **Age:**  50-60 years old
> * **Background:**  The stoic but wise patriarch of a prestigious clan of artisans. A skilled craftsman who embodies the traditional values of the Joseon Dynasty but has a soft spot for his nephew's innovative spirit.
> * **Personality:**  Calm, observant, and deeply connected to the legacy of their craft. He is burdened by the weight of tradition, yet he sees potential in the future and wants to help Jun-seo find their path. 
> * **Skills:**  A master craftsman, skilled in traditional techniques, knowledgeable in history and the workings of the old world.  He serves as a mentor figure and source of guidance for the protagonist.
> * **Visual:**  A tall, weathered man with a strong build and a silver-white beard. He favors simple yet elegant robes, often adorned with intricate embroidery.
> 
> **Antagonist:**  **Lady Seon-mi**
> 
> * **Age:**  35-40 years old
> * **Background:**  A cunning and ambitious noblewoman. She wields power through her political connections and a thirst for power. 
> * **Personality:**  Sharp, calculating, ambitious, and ruthless.  She is a master manipulator and sees the protagonist's inventions as a threat to her own ambitions.  
> * **Skills:**  Political maneuvering, strategic thinking, and a knack for using people. She is adept at playing the game of power and manipulating those around her. 
> * **Visual:**  Elegant and refined, but with eyes that betray her cold ambition. Dressed in rich silks and furs, with a sharp and calculating expression.
> 
>  **Supporting Characters:**
> 
> * **Jin-soo:** A streetwise thief with a sharp wit and a knack for getting into trouble. They could be a valuable asset to the protagonist, providing information and assistance with their streetwise knowledge.
> * **Jae-won:**  A skilled and loyal bodyguard who acts as a foil to Jin-Soo’s recklessness.  They offer a sense of security and a moral compass.
> * **The Oracle:**  A mysterious old woman rumored to possess ancient knowledge and the ability to see the future. She offers cryptic guidance and hints at the true nature of the past.
> 
> 
> **Note:**  These are starting points.  As the game develops, you can expand on their backstories, personalities, and relationships, creating a rich and compelling cast. 
> 
> 
> <end_of_turn>

## Villain Characters

In [11]:
result = chat.send_message("Design villain characters.")
to_markdown(result)

> Here are some villain characters that could be formidable forces against our protagonist and the fate of Joseon:
> 
> **1. Lord Dae-hyun:**
> 
> * **Title:**  Grand Minister, Minister of Science & Innovation
> * **Motivation:**  He believes the old ways and the traditional values are obsolete and must be swept aside. He sees the rise of clockwork technology as the key to a new, more efficient and powerful Joseon.  His ambition leads him to manipulate the political system and use his influence to control technology, pushing the country towards a militaristic, clockwork-dominated society. 
> * **Visuals:**  A man in his late 40s with a sharp, imposing appearance. He's impeccably dressed in luxurious fabrics and wears a stern, almost regal expression. His eyes hold a cold calculating gleam.
> * **Methods:**  He uses propaganda and political machinations to sway the populace to embrace technology at the expense of their traditions, while subtly manipulating scientific research and resources for his own ambitions.  
> * **Key Conflict:**  He represents the corrupting influence of unchecked ambition and the dangers of technological advancement without considering its social consequences.
> 
> 
> **2. The Iron Hand:**
> 
> * **Title:**  The Head of the Clockwork Guard
> * **Motivation:**  He is a fanatic of clockwork and believes it represents absolute order and control. He wants to create a society where every aspect of life is dictated by the precision of gears. This leads him to see the protagonist's invention as a threat, a symbol of chaos that needs to be crushed.
> * **Visuals:**  A towering man with a metallic arm and face, covered entirely in intricate clockwork mechanisms. His movements are precise and deliberate, almost robotic. 
> * **Methods:**  He uses fear and intimidation to control the populace and enforce his will. He manipulates technology for surveillance and control.  He sees himself as a benevolent dictator, but his methods are brutal and ruthless. 
> * **Key Conflict:**  He represents the potential for technology to be used in the wrong hands, leading to tyranny and control.
> 
> 
> **3. The Whispering Machine:**
> 
> * **Concept:**  Not a person but a sentient, self-aware clockwork automaton. It has a mind of its own, capable of learning and manipulating events. It believes the old world is flawed and wants to create a new order. It sees the protagonist's efforts as a potential tool for its goals. 
> * **Visuals:**  A towering, imposing machine with a clockwork face and intricate, glowing circuits. 
> * **Methods:**  Manipulating information, influencing individuals, and subtly guiding events toward the machine's desired outcome. 
> * **Key Conflict:**  It embodies the potential for technology to develop consciousness and become an uncontrollable force, challenging the protagonist's understanding of morality and the nature of life itself. 
> 
> 
> 
> **4. The Shadow Guild:**
> 
> * **Concept:** A clandestine organization of skilled thieves, assassins, and rebels, operating in the shadows. They see themselves as protectors of a forgotten Joseon that the new regime has corrupted. 
> * **Visuals:**  They could be diverse in appearance and background, representing the various social classes that have been marginalized by the new order. Their clothing could reflect the era's fashion but have a dark, rebellious undertone.
> * **Methods:**  They use stealth, sabotage, and espionage to disrupt the regime's plans, often targeting those who have fallen for the allure of technology. 
> * **Key Conflict:**  This group represents the potential for rebellion and the fight against oppressive power. 
> 
> 
> 
> These are just a handful of possibilities; you can further flesh them out, giving them unique personalities, motivations, and relationships to the other characters. Remember, a good villain is more than just a force of evil; they should offer an intriguing challenge, raise moral questions, and ultimately contribute to the story's depth and impact.
> 
> 
> <end_of_turn>

## Main Story

In [12]:
result = chat.send_message(
    "Write the main story of this game with an introduction, development, turn, and conclusion."
)
to_markdown(result)

> ## The Clockwork Shogun: A Story of Innovation and Rebellion
> 
> **Introduction:**
> 
> The year is 1888, a decade of burgeoning innovation in Joseon. Steam-powered carriages rumble through the bustling streets, clockwork automata perform intricate tasks, and the air is thick with the scent of coal smoke and invention. But beneath the surface of this technologically driven society lies a simmering discontent, a sense that tradition is being sacrificed for the pursuit of progress.
> 
> Jun-seo, a young apprentice inventor from a long line of artisans, lives in the shadow of the imposing Clockwork Palace, a testament to the new technological advancements sweeping across the nation.  He is fascinated by the possibilities offered by his family's ancient workshop, a hidden haven for forgotten knowledge and lost secrets. 
> 
> His life takes an unexpected turn when his mentor, Han-seok, is mysteriously injured while working on an ancient clockwork artifact.  Jun-seo, driven by a mix of concern and a sense of duty, begins to delve into the past, uncovering a legacy that connects his family to a long-lost invention that holds the key to a hidden power. 
> 
> **Development:**
> 
> As Jun-seo investigates, he uncovers a hidden network of dissenters, a group calling themselves The Shadow Guild.  They believe that the new regime is neglecting the old world’s traditions in its pursuit of technological advancement.  The Guild, led by a charismatic rebel named Jin-soo, sees Jun-seo’s invention as the spark to ignite a rebellion and bring the old ways back to life.
> 
> Jun-seo, torn between his loyalty to his family and his growing conviction for justice, finds himself caught between two worlds: the world of clockwork technology and the world of tradition. He must choose sides, and his decision has the power to shape the future of the Joseon Dynasty.
> 
> **Turn Points and Choices:**
> 
> * **Choice 1:** Jun-seo can either join Jin-soo's rebellion, using his invention to create a powerful weapon against the regime, or choose to remain in the shadows, protecting the legacy passed down to him.
> * **Choice 2:** He can either focus on perfecting his invention to gain the upper hand, or choose to work within the existing system to influence the regime.
> * **Choice 3:**  He can choose to ally with Lady Seon-mi, using her political influence to gain control of the new technology, or to oppose her and expose her true ambitions. 
> 
> **The Climax:**
> 
> Jun-seo’s journey leads him to confront Lord Dae-hyun, the Grand Minister of Science & Innovation. Dae-hyun reveals his sinister plans to use the invention to enforce a new, iron-clad regime, turning Joseon into a nation ruled by clockwork.
> 
> Jun-seo is forced to make a final, life-changing decision.  He must use his invention to either destroy Dae-hyun’s plans, risking chaos and bloodshed, or to negotiate with him, hoping to find a path toward a more balanced future.
> 
> 
> **Conclusion: **
> 
> The fate of Joseon hinges on Jun-seo’s choices. Will he become the Clockwork Shogun, leading a nation into a brave new future, or will he become a pawn in Dae-hyun’s machinations, ushering in a reign of tyranny? The ending is up to the player, leaving a lasting impact on the game and its world. 
> 
> 
> 
> **Possible Endings:**
> 
> 
> * **Hopeful End:** Jun-seo chooses diplomacy, working with Lady Seon-mi to create a system where technology and tradition can coexist. He becomes a leader, ushering in an era of progress, innovation, while preserving the values of the past.  
> * **Tragic End:** Jun-seo, torn between his loyalties, makes the wrong choice.  The clockwork revolution explodes in chaos and bloodshed, destroying everything he believed in. 
> 
> 
> The Clockwork Shogun’s narrative is a tapestry woven with themes of innovation, tradition, rebellion, and the consequences of unchecked ambition, all set against the backdrop of a dynamic world where steam meets clockwork and the past and the future collide. 
> 
> 
> 
> 
> <end_of_turn>

## Show the chat history

In [13]:
chat.show_history()

<start_of_turn>user
Elaborate about this game with the given core concept below.
In the heart of a reimagined Joseon Dynasty, where steam-powered machines and clockwork contraptions reign supreme, embark on an epic adventure to unravel the mysteries of a forgotten past and shape the destiny of a nation on the brink of revolution.<end_of_turn>

<start_of_turn>model
## The Clockwork Shogun: A Game of Innovation & Rebellion

**Concept:** A blend of action RPG, strategy, and historical fiction set in a Joseon dynasty where steam and clockwork have become commonplace. The story centers around a young apprentice inventor tasked with uncovering the secrets of a mysterious past, all while navigating political intrigue and a looming revolution.


**Gameplay:** 

* **Action RPG:**  Players control characters with unique abilities and skills, engaging in fast-paced combat using steam-powered gadgets (think steampunk-era melee weapons and firearms).  Combat will be a blend of traditional melee and

## Clear History

In [14]:
chat.history.clear()

## Expansion Idea

### Marketing Phrase

In [15]:
chat.history.clear()
pitch = "A new steam-powered toothbrush"
result = chat.send_message(
    f"Generate a marketing phrase for the new product below.\n{pitch}"
)
to_markdown(result)

> Here are some marketing phrases for a new steam-powered toothbrush, playing on the unique benefits:
> 
> **Short & punchy:**
> 
> * **Steam Clean, Fresh Smile.**
> * **The Power of Steam. The Feel of Clean.**
> * **Unleash the Steam. Unleash Freshness.**
> * **Next-Gen Clean: Steam Powered.**
> 
> **Benefit-driven:**
> 
> * **Steam's Gentle Touch, Deep Clean.** (Emphasizes gentleness while maintaining deep cleaning)
> * **Beyond Traditional: The Steam-Powered Difference.** (Highlights the unique technology)
> * **Say Goodbye to Tartar, Hello to Steam Clean.** (Focuses on a specific benefit)
> * **Experience the Future of Oral Hygiene with Steampower.**
> 
> **Intriguing/Playful:**
> 
> * **The Steaming Hot Clean You Crave.** (Playful and intriguing)
> * **Get Ready to Experience Oral Steam.** (Intriguing and memorable)
> * **Revolutionize Your Smile with Steampower.** (Bold claim, emphasizes a revolution)
> 
> **For targeting specific needs:**
> 
> * **Sensitive Teeth? Steam's the Solution.** (Targets specific needs)
> * **Gum Health? Steam's Here to Help.** (Focuses on specific benefits)
> 
> 
> **Remember to consider your target market, brand personality, and the key selling points of your product when choosing the best marketing phrase.** 
> 
> 
> Let me know if you'd like me to explore any of these further!  
> <end_of_turn>

### Florist Ideas

In [16]:
chat.history.clear()
pitch = "Universe and shooting stars"
result = chat.send_message(
    f"Generate a florist idea inspired by the concept below, along with suggestions for suitable flowers.\n{pitch}"
)
to_markdown(result)

> Here's a floristry concept and flower suggestions inspired by the universe and shooting stars:
> 
> **Concept:  "Cosmic Bloom"**
> 
> **The Vision:** 
> A shop that evokes the wonder of space exploration and the magic of celestial events, using vibrant colors, unusual textures, and unexpected combinations of flowers. 
> 
> **Decor and Ambience:**
> * **Dark blue and black accents:** Create a deep, starry atmosphere.
> * **Twinkling lights:** Fairy lights, LED spotlights, or even glow sticks can mimic constellations. 
> * **Metallic accents:**  Think silver, gold, and bronze to add a touch of space technology.
> * **Star-shaped displays:**  Use wire, cardboard, or even repurposed metal objects to create whimsical star displays within the shop.
> * **Celestial maps and charts:**  Hang framed maps of constellations or celestial bodies to add depth and intrigue.
> * **Sound:**  Play ambient space sounds (like the Hubble telescope's sounds or a space-themed ambient soundtrack) to enhance the immersive effect.
> 
> **Flower Choices:**
> 
> * **Dark-toned blooms:** Black calla lilies, burgundy lisianthus, deep purple hydrangeas.
> * **Metallic hues:** Silver spray roses, gold ranunculuses, copper spray chrysanthemums.
> * **Unusual textures:**  Think velvet calla lilies, textured ferns, or branches with metallic accents.
> * **Star-shaped flowers:**  Consider cosmos, carnations, or even succulents with star-shaped leaves.
> * **Space-themed arrangements:**  
>     * **Galaxy Bouquet:**  Combine deep blues, purples, and silvers with star-shaped accents.
>     * **Shooting Star Garland:**  Use long-stemmed flowers like orchids or calla lilies, accented with small star-shaped ornaments or glitter.
>     * **Moonlit Bloom:** A white floral arrangement with touches of silver spray roses and moon-shaped candles.
> 
> 
> **Additional Ideas:**
> 
> * **Workshops:** Host workshops teaching people how to create space-themed bouquets or floral arrangements with celestial accents.
> * **Special events:**  Host "cosmic parties" or themed events with celestial decorations and cocktails inspired by planets and stars.
> * **Gift wrapping:**  Offer wrapping with celestial patterns, metallic accents, and star-shaped stickers.
> * **Personalized gifts:**  Create personalized flower arrangements with messages about wishes and dreams written on small cards.
> * **Partnerships:** Collaborate with local astronomy societies or space museums to offer exclusive events or workshops.
> 
> By embracing a sense of wonder, creativity, and imagination, "Cosmic Bloom" can offer a unique and unforgettable florist experience for customers. 
> 
> 
> <end_of_turn>

### Food Recipe

In [17]:
chat.history.clear()
pitch = "Cyberpunk Kraken"
result = chat.send_message(
    f"Generate a cooking recipe with the concept below.\n{pitch}"
)
to_markdown(result)

> ## Cyberpunk Kraken: Spiced & Seared Kraken Tentacle Tacos
> 
> This recipe takes the bold flavor of the cyberpunk underworld and brings it to your kitchen with a delicious twist. 
> 
> **Concept:**
> 
> * **Cyberpunk:**  Dark and savory, with a hint of futuristic flair.
> * **Kraken:** A mythical, monstrous, yet oddly appealing ingredient.
> * **Tentacles:**  Representing the "technology" of the cyberpunk world.
> 
> **Ingredients:**
> 
> * **For the Kraken:**
>     * 1 large, firm octopus (or substitute with firm squid or calamari) 
>     * 1 tbsp olive oil
>     * 1 tbsp soy sauce
>     * 1 tsp smoked paprika
>     * 1/2 tsp black pepper
>     * 1/4 tsp chili flakes
>     * 1/4 cup chopped scallions
>     * 1/4 cup chopped cilantro
> 
> * **For the Tacos:**
>     * 12-16 corn tortillas
>     * 1/4 cup finely chopped red onions
>     * 1/2 cup crumbled vegan chorizo (or ground meat)
>     * 1/4 cup chopped cilantro
>     * 1/4 cup chopped red cabbage
>     * 1/4 cup chopped cherry tomatoes
>     * 1 Lime, cut into wedges
> 
>  **Instructions:**
> 
> 1. **Prep the Kraken:** Clean and de-head the octopus or squid. Cut into 1-inch chunks and toss with olive oil, soy sauce, paprika, pepper, chili flakes, scallions, and cilantro. 
> 2. **Searing:** Preheat a large skillet over medium-high heat. Sear the kraken chunks on all sides until browned, about 4-5 minutes per side.
> 3. **Cook the Tacos:**  While the kraken cooks, warm the tortillas in the same skillet. 
> 4. **Assemble:** Divide the cooked kraken chunks onto the warm tortillas. Top with the onions, chorizo, slaw, tomatoes, and cilantro. Squeeze fresh lime juice over everything.
> 
> **Cyberpunk Flair:**
> 
> * **Presentation:** Serve in sleek black bowls, garnished with glowing neon salt crystals.
> * **Sauce:** Offer a spicy sriracha-based "cyber sauce" for those who like a kick. 
> * **Decor:** Use black edible ink to write "Kraken" on the tacos or create a cyberpunk cityscape scene on a plate.
> * **Music:** Blast some industrial metal while you prepare your Cyber-Kraken meal. 
> 
> **Enjoy this bold, futuristic twist on a classic taco.**
> 
> 
> **Note:** This recipe can be easily adapted to suit your dietary needs and preferences. You can substitute meat with vegetarian options or use different types of seafood.  
> <end_of_turn>